In [12]:
# модуль Python, который предоставляет общие математические и числовые операции 
import numpy as np
# библиотека scipy.special с сигмоидой expit()
import scipy.special
#библиотека для графического отображения массивов
import matplotlib.pyplot
#гарантировать размещение графики в данном блокноте, а не в отдельном окне
%matplotlib inline
#измерение времени выполнения обучения нейросети
import time

In [13]:
# помогает загрузить данные из изображений формата PNG
import imageio
# glob помогает выбрать несколько файлов, используя шаблон для выбора
import glob

<b>Создание класса нейросети</b>

In [14]:
# определение класса нейронной сети
class neuralNetwork:
    
    #инициализация - задание количества входных, скрытых и выходных узлов
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # задание количества узлов во входном, скрытом и выходном слое
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # Матрицы весовых коэффициентов связей wih (между входным и скрытым 
        # слоями) и who (между скрытым и выходным слоями).
        # Весовые коэффициенты связей между узлом i и узлом j следующего слоя
        # обозначены как w_i_j:
        # w11 w21
        # w12 w22 и тд.
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), 
                                      (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), 
                                      (self.onodes, self.hnodes))
        
        # коэффициент обучения
        self.lr = learningrate
        
        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # тренировка нейронной сети - уточнение весовых коэффициентов в процессе 
    # обработки предоставленных для обучения сети тренировочных примеров
    def train(self, inputs_list, targets_list):
        # преобразовать список входных значений в двухмерный массив
        inputs = np.array(inputs_list, ndmin = 2).T
        targets = np.array(targets_list, ndmin = 2).T
        
        # расчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # расчитать исходящие сигналы для скрытыго слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # расчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # расчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        # ошибки выходного слоя = 
        # (целевое значение - фактическое значение)
        output_errors = targets - final_outputs
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # обновить весовые коэффициенты для связей между
        # скрытым и выходным слоями
        self.who += self.lr * np.dot((output_errors * final_outputs *
                                     (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # обновить весовые коэффициенты для связей между
        # входным и скрытым слоями
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs *
                                     (1.0 - hidden_outputs)), np.transpose(inputs))
        pass
    
    # опрос нейронной сети - получение значений сигналов с выходных узлов 
    # после предоставления значений входящих сигналов
    def query(self, inputs_list):
        # преобразовать список входных значений 
        # в двумерный массив
        inputs = np.array(inputs_list, ndmin = 2).T
        
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    

<b>Инициализация нейросети</b>

In [15]:
# количество входных, скрытых и выходных узлов

#Число представляет собой количество пикселей, из которых состоит изображение цифры
input_nodes = 784
#Число не больше 784, так как сеть должна находить во входных значениях такие шаблоны,
#которые можно выразить в более короткой форме, чем сами эти значения.
hidden_nodes = 200
#Выходной слой должен обеспечивать вывод 10 маркеров, значит, должен иметь 10 узлов.
output_nodes = 10

# коэффициент обучения равен 0,2
learning_rate = 0.2

# создание экземпляра нейронной сети
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

<b>Тренировка нейросети</b>

In [16]:
#загрузить в список тестовый набор данных CSV-файла набора MNIST
training_data_file = open('mnist_train.csv', 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

#переменная указывает, сколько раз тренировочный набор данных
#используется для тренировки сети
epochs = 8

#фиксация начала обучения нейросети
start_time = time.time()
for e in range(epochs):
    #перебрать все записи в тренировочном наборе данных
    for record in training_data_list:
        #получить список значений, используя символы запятой в качестве разделителей
        all_values = record.split(',')
        #масштабировать и сместить входные значения
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        #дополнительное масштабирование выходных значений - создание целевых выходных значений -
        #все равны 0.01, за исключением желаемого маркерного значения 0.99
        targets = np.zeros(output_nodes) + 0.01
    
        #all_values[0] - целевое маркерное значение для данной записи
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        
#Вывод продолжительности обучения
print("Network trained {} minutes".format(str(round((time.time() - start_time)/60, 1))))

Network trained 6.1 minutes


<b>Загрузка данных для распознавания</b>

In [17]:
# собственный набор данных для тестирования изображений
work_dataset = []
items = 0
for image_file_name in glob.glob('my_dataset/??_2828_?num.png'):
    print ("loading ... ", image_file_name)
    # использование имени файла, чтобы установить правильный маркер для изображения
    label = int(image_file_name[-8:-7])
    # загрузить данные изображения PNG в массив
    img_array = imageio.imread(image_file_name, as_gray=True)
    # преобразование квадратного массива, размерностью 28x28 в длинный список значений
    img_data  = 255.0 - img_array.reshape(784)
    # Масштабирование данных от 0.01 до 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    print(np.min(img_data))
    print(np.max(img_data))
    # добавляем в конец списка маркер и данные по изображению PNG
    record = np.append(label,img_data)
    work_dataset.append(record)
    items += 1

loading ...  my_dataset/06_2828_8num.png
0.01
1.0
loading ...  my_dataset/08_2828_8num.png
0.01
1.0
loading ...  my_dataset/07_2828_4num.png
0.01
1.0
loading ...  my_dataset/02_2828_0num.png
0.01
1.0
loading ...  my_dataset/09_2828_0num.png
0.01
1.0
loading ...  my_dataset/03_2828_3num.png
0.01
1.0
loading ...  my_dataset/00_2828_1num.png
0.01
1.0
loading ...  my_dataset/10_2828_0num.png
0.01
1.0
loading ...  my_dataset/01_2828_0num.png
0.01
1.0
loading ...  my_dataset/04_2828_4num.png
0.01
1.0
loading ...  my_dataset/05_2828_1num.png
0.01
1.0


<b>Опрос нейросети по загруженным данным</b>

In [18]:
# опрос нейросети по каждому кадру
for item in range(items):
    # печать кадра, передаваемого сети - проигнорировать первое значение, являющееся маркером, 
    # извлечь оставшиеся 28*28 = 784 значения
    # и преобразовать их в массвив, состоящий из 28 строк и 28 столбцов
    #matplotlib.pyplot.imshow(work_dataset[item][1:].reshape(28,28), cmap='Greys', interpolation='None')
    
    # правильный ответ - первое значение
    correct_label = work_dataset[item][0]
    # на ввод принимаются все данные по изображению, кроме маркера
    inputs = work_dataset[item][1:]
    
    #опрос сети
    outputs = n.query(inputs)
    #print(outputs)
    # Наибольшее из значений входных узлов рассматривается сетью в качестве правильного ответа
    #=> индекс наибольшего значения является маркерным значением 
    label = np.argmax(outputs)
    print("network says ", label)
    print("correct answer", correct_label)
    # узнать правильность ответа
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!")

network says  4
correct answer 8.0
no match!
network says  4
correct answer 8.0
no match!
network says  4
correct answer 4.0
match!
network says  0
correct answer 0.0
match!
network says  0
correct answer 0.0
match!
network says  3
correct answer 3.0
match!
network says  1
correct answer 1.0
match!
network says  0
correct answer 0.0
match!
network says  0
correct answer 0.0
match!
network says  4
correct answer 4.0
match!
network says  1
correct answer 1.0
match!
